## Importing Libraries

In [1]:
import argparse
import datetime as dte
import os
import json

import data_formatters.base
import expt_settings.configs
import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf

import warnings 

warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

ExperimentConfig = expt_settings.configs.ExperimentConfig
HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer
tf.experimental.output_all_intermediates(True)

### Experiment Metrics Tracking

In [2]:
with open('traffic_dataset_experiments.json', 'r') as f:
    loss_experiment_tracker = json.load(f)

dataset_name = "traffic"
dataset_folder_path = "traffic_dataset"

### Creating Model

In [3]:
loss_experiment_tracker[dataset_folder_path] = {}

name = dataset_name
output_folder = dataset_folder_path

use_tensorflow_with_gpu = True
print("Using output folder {}".format(output_folder))

config = ExperimentConfig(name, output_folder)
formatter = config.make_data_formatter()

expt_name=name
use_gpu=use_tensorflow_with_gpu
model_folder=os.path.join(config.model_folder, "fixed")
data_csv_path=config.data_csv_path
data_formatter=formatter
use_testing_mode=True

num_repeats = 1

if not isinstance(data_formatter, data_formatters.base.GenericDataFormatter):
    raise ValueError(
        "Data formatters should inherit from" +
        "AbstractDataFormatter! Type={}".format(type(data_formatter)))

# Tensorflow setup
default_keras_session = tf.keras.backend.get_session()

if use_gpu:
    tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
    tf_config = utils.get_default_tensorflow_config(tf_device="cpu")

print("*** Training from defined parameters for {} ***".format(expt_name))

print("Loading & splitting data...")
raw_data = pd.read_csv(data_csv_path, index_col=0)
train, valid, test = data_formatter.split_data(raw_data)
train_samples, valid_samples = data_formatter.get_num_samples_for_calibration()

# Sets up default params
fixed_params = data_formatter.get_experiment_params()
params = data_formatter.get_default_model_params()
params["model_folder"] = model_folder

# Parameter overrides for testing only! Small sizes used to speed up script.
if use_testing_mode:
    fixed_params["num_epochs"] = 1
    params["hidden_layer_size"] = 5
    train_samples, valid_samples = 100, 10

# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
                                   fixed_params, model_folder)

# Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")

for k in params:
    print("{}: {}".format(k, params[k]))

best_loss = np.Inf

for _ in range(num_repeats):

    tf.reset_default_graph()
    with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

        tf.keras.backend.set_session(sess)

        params = opt_manager.get_next_parameters()
        model = ModelClass(params, use_cudnn=use_gpu)

        if not model.training_data_cached():
            model.cache_batched_data(train, "train", num_samples=train_samples)
            model.cache_batched_data(valid, "valid", num_samples=valid_samples)

        sess.run(tf.global_variables_initializer())
        model.fit()

        val_loss = model.evaluate()

        if val_loss < best_loss:
            opt_manager.update_score(params, val_loss, model)
            best_loss = val_loss

        tf.keras.backend.set_session(default_keras_session)

Using output folder traffic_dataset

Metal device set to: Apple M1 Pro
Selecting GPU ID=0
*** Training from defined parameters for traffic ***
Loading & splitting data...
Formatting train-valid-test splits.
Setting scalers with training data...
*** Loading hyperparm manager ***
*** Running calibration ***
Params Selected:
dropout_rate: 0.3
hidden_layer_size: 5
learning_rate: 0.001
minibatch_size: 128
max_gradient_norm: 100.0
num_heads: 4
stack_size: 1
model_folder: traffic_dataset/saved_models/traffic/fixed
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), (

                                                                  'multiply[0][0]']               
                                                                                                  
 multiply_1 (Multiply)          (None, 1, 5)         0           ['dense_11[0][0]',               
                                                                  'dense_12[0][0]']               
                                                                                                  
 layer_normalization (LayerNorm  (None, 1)           2           ['add[0][0]']                    
 alization)                                                                                       
                                                                                                  
 add_1 (Add)                    (None, 1, 5)         0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/strided_slice_11[0][0]',  
          

 tf_op_layer_TemporalFusionTran  [(None, 192, 5, 1)]  0          ['time_distributed[0][0]']       
 sformer/stack_1 (TensorFlowOpL                                                                   
 ayer)                                                                                            
                                                                                                  
 multiply_3 (Multiply)          (None, 5)            0           ['dense_15[0][0]',               
                                                                  'dense_16[0][0]']               
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 3)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_8 (Tenso                                  former/stack_2[0][0]']           
 rFlowOpLayer)                                                                                    
          

 ibuted)                                                         former/strided_slice_12[0][0]']  
                                                                                                  
 time_distributed_14 (TimeDistr  (None, 168, 5)      30          ['tf_op_layer_TemporalFusionTrans
 ibuted)                                                         former/strided_slice_13[0][0]']  
                                                                                                  
 time_distributed_18 (TimeDistr  (None, 168, 5)      30          ['tf_op_layer_TemporalFusionTrans
 ibuted)                                                         former/strided_slice_14[0][0]']  
                                                                                                  
 time_distributed_22 (TimeDistr  (None, 168, 5)      30          ['tf_op_layer_TemporalFusionTrans
 ibuted)                                                         former/strided_slice_15[0][0]']  
          

 dropout_9 (Dropout)            (None, 168, 5)       0           ['time_distributed_19[0][0]']    
                                                                                                  
 dropout_10 (Dropout)           (None, 168, 5)       0           ['time_distributed_23[0][0]']    
                                                                                                  
 dense_21 (Dense)               (None, 5)            30          ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]']               
                                                                                                  
 dense_25 (Dense)               (None, 5)            30          ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]']               
                                                                                                  
 dropout_1

...

 ibuted)                                                                                          
                                                                                                  
 time_distributed_50 (TimeDistr  (None, 192, 5)      30          ['dropout_16[0][0]']             
 ibuted)                                                                                          
                                                                                                  
 multiply_19 (Multiply)         (None, 192, 5)       0           ['time_distributed_49[0][0]',    
                                                                  'time_distributed_50[0][0]']    
                                                                                                  
 add_16 (Add)                   (None, 192, 5)       0           ['layer_normalization_15[0][0]', 
                                                                  'multiply_19[0][0]']            
          

                                                                 former/Cumsum[0][0]']            
                                                                                                  
 lambda_6 (Lambda)              (None, 192, 192)     0           ['dense_81[0][0]',               
                                                                  'dense_82[0][0]']               
                                                                                                  
 lambda_7 (Lambda)              (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 lambda_9 (Lambda)              (None, 192, 192)     0           ['dense_83[0][0]',               
                                                                  'dense_84[0][0]']               
          

 multiply_20 (Multiply)         (None, 192, 5)       0           ['time_distributed_51[0][0]',    
                                                                  'time_distributed_52[0][0]']    
                                                                                                  
 add_21 (Add)                   (None, 192, 5)       0           ['multiply_20[0][0]',            
                                                                  'layer_normalization_16[0][0]'] 
                                                                                                  
 layer_normalization_17 (LayerN  (None, 192, 5)      10          ['add_21[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 time_distributed_53 (TimeDistr  (None, 192, 5)      30          ['layer_normalization_17[0][0]'] 
 ibuted)  

Cached data "train" updated
Getting valid sampling locations.
Getting locations for 400000
Getting locations for 400001
Getting locations for 400009
Getting locations for 400010
Getting locations for 400015
Getting locations for 400017
Getting locations for 400025
Getting locations for 400026
Getting locations for 400027
Getting locations for 400030
Getting locations for 400031
Getting locations for 400035
Getting locations for 400037
Getting locations for 400039
Getting locations for 400040
Getting locations for 400041
Getting locations for 400043
Getting locations for 400044
Getting locations for 400045
Getting locations for 400049
Getting locations for 400052
Getting locations for 400053
Getting locations for 400057
Getting locations for 400059
Getting locations for 400060
Getting locations for 400065
Getting locations for 400067
Getting locations for 400071
Getting locations for 400073
Getting locations for 400074
Getting locations for 400075
Getting locations for 400078
Getting lo

*** Fitting TemporalFusionTransformer ***
Getting batched_data
Using cached training data
Using cached validation data
Using keras standard fit
Train on 100 samples, validate on 10 samples
100/100 [==============================] - 10s 101ms/sample - loss: 1.8994 - val_loss: 1.7151
Cannot load from traffic_dataset/saved_models/traffic/fixed/tmp, skipping ...
Using cached validation data
Optimal model found, updating
INFO:tensorflow:traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt.index
INFO:tensorflow:0
INFO:tensorflow:traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt.meta
INFO:tensorflow:4300
INFO:tensorflow:traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt.data-00000-of-00001
INFO:tensorflow:4300
Model saved to: traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt


## Calculating Mean Absolute Error ( MAE )

In [4]:
from sklearn.metrics import mean_absolute_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_absolute_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

    
print("*"*100)
print("*"*100)
print("Mean Absolute Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Absolute Error p10 Loss": str(p10_loss.mean()),
    "Mean Absolute Error p50 Loss": str(p50_loss.mean()),
    "Mean Absolute Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[[-0.29136884  0.50544816  0.2864225  -0.76421726  0.44517332]
 [-0.16917744 -0.3214742   0.76211846  0.7094366   0.08697722]
 [-0.5189309  -0.5664482  -0.5703417   0.05330397  0.1094504 ]
 [ 0.01542873 -0.02704871 -0.53360415 -0.20114477 -0.01259346]
 [ 0.10385615 -0.45283782 -0.4925069   0.7725902  -0.20208116]]
tensor: TemporalFusionTransformer/dense_23/bias (float32) [5]
[-0.00040042  0.00061337  0.00062569 -0.00030751 -0.00063063]
tensor: TemporalFusionTransformer/dense_23/kernel (float32) [5, 5]
[[-0.29112116  0.50494856  0.28592956 -0.7643629   0.44603792]
 [-0.16907376 -0.32157576  0.76240283  0.7097422   0.08725736]
 [-0.51913244 -0.5668373  -0.57028043  0.05386046  0.11047085]
 [ 0.01483628 -0.02669022 -0.532771   -0.2007146  -0.01274248]
 [ 0.10367888 -0.45259407 -0.49228925  0.7727026  -0.20220518]]
tensor: TemporalFusionTransformer/dense_24/bias (float32) [5]
[-1.57208167e-04 -1.02130405e-04 -1.59270639e-05  2.77531799e-05
  4.14705719e-04]
tensor: TemporalFusionTransforme

[[-0.29256797  0.50656456  0.28552064 -0.7632275   0.44637755]
 [-0.17040133 -0.32025704  0.7614773   0.708709    0.08810595]
 [-0.5205019  -0.56529665 -0.5706405   0.05488619  0.11084017]
 [ 0.01403511 -0.02607463 -0.5342239  -0.19989592 -0.0112293 ]
 [ 0.10282198 -0.45181695 -0.49327073  0.77376074 -0.20117229]]
tensor: TemporalFusionTransformer/time_distributed_4/bias (float32) [4]
[ 0.00099336 -0.00099613  0.00099562 -0.0009917 ]
tensor: TemporalFusionTransformer/time_distributed_4/kernel (float32) [20, 4]
[[-0.1872162   0.3253476   0.18593574 -0.49427196]
 [ 0.288401   -0.10835505 -0.20836283  0.49121517]
 [ 0.45712695  0.05524413 -0.33434469 -0.36476237]
 [-0.36678693  0.03381721  0.0718817   0.01069185]
 [-0.01847126 -0.3431983  -0.1306716  -0.00690433]
 [ 0.06607254 -0.29328784 -0.31676498  0.49981037]
 [-0.12950732 -0.1197278   0.27131608  0.181257  ]
 [-0.10162049  0.28506708  0.17781319  0.23296662]
 [ 0.05105373 -0.3895213   0.14780824  0.49005866]
 [ 0.31946108  0.20370983

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-24 23:50:09.392499
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Mean Bias Error (MBE)

In [5]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = np.mean(extract_numerical_data(p10_forecast)-extract_numerical_data(targets))
    p50_loss = np.mean(extract_numerical_data(p50_forecast)-extract_numerical_data(targets))
    p90_loss = np.mean(extract_numerical_data(p90_forecast)-extract_numerical_data(targets))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Bias Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Bias Error p10 Loss": str(p10_loss.mean()),
    "Mean Bias Error p50 Loss": str(p50_loss.mean()),
    "Mean Bias Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[ 0.00015795  0.00045845 -0.00072721  0.00051106  0.00045886]
tensor: TemporalFusionTransformer/dense_28/kernel (float32) [5, 5]
[[-0.2918945   0.50529593  0.28721112 -0.7646856   0.4448742 ]
 [-0.16960818 -0.3211609   0.7632186   0.7095658   0.08649015]
 [-0.5189834  -0.5666003  -0.5689789   0.05386044  0.1093748 ]
 [ 0.01543139 -0.02727989 -0.5332826  -0.20116557 -0.01195911]
 [ 0.10403006 -0.45267755 -0.49255213  0.77308345 -0.20204258]]
tensor: TemporalFusionTransformer/dense_4/bias (float32) [1]
[0.]
tensor: TemporalFusionTransformer/dense_4/kernel (float32) [5, 1]
[[-0.37641406]
 [ 0.65268254]
 [ 0.36989117]
 [-0.9865818 ]
 [ 0.5749903 ]]
tensor: TemporalFusionTransformer/dense_5/bias (float32) [5]
[0. 0. 0. 0. 0.]
tensor: TemporalFusionTransformer/dense_5/kernel (float32) [5, 5]
[[-0.29156908  0.50556576  0.28651643 -0.764203    0.44538558]
 [-0.1694026  -0.3212554   0.76246536  0.7096921   0.08711606]
 [-0.5195019  -0.56629664 -0.5696406   0.05392718  0.10984051]
 [ 0.01503503 

[[-0.326983    0.5642402   0.3193363 ]
 [-0.8534052   0.49895543 -0.18839854]
 [-0.35817537  0.8534605   0.7944581 ]
 [ 0.09639957 -0.57982177 -0.6341385 ]
 [-0.63587993  0.06129171  0.12378685]]
tensor: TemporalFusionTransformer/time_distributed_32/bias (float32) [5]
[-0.00099983  0.00099968  0.00099998  0.00099979  0.00099992]
tensor: TemporalFusionTransformer/time_distributed_32/kernel (float32) [5, 5]
[[-0.29256865  0.5065575   0.28551653 -0.7652026   0.44438708]
 [-0.16840284 -0.3222506   0.7634653   0.71069187  0.08811521]
 [-0.51850235 -0.5672882  -0.56864065  0.05492675  0.11083899]
 [ 0.01403518 -0.02607772 -0.5342241  -0.20189504 -0.01322776]
 [ 0.10482047 -0.4538105  -0.49127355  0.7737953  -0.20116156]]
tensor: TemporalFusionTransformer/time_distributed_33/bias (float32) [5]
[-0.00099999 -0.00099978 -0.00099993  0.00099986 -0.00099984]
tensor: TemporalFusionTransformer/time_distributed_33/kernel (float32) [5, 5]
[[-0.29056942  0.5065623   0.2875158  -0.76518303  0.44438612]

[[-0.30639103  0.53382695  0.30107093 -0.80644995]
 [ 0.4704494  -0.17758597 -0.3396229   0.8027338 ]
 [ 0.74903774  0.0910331  -0.54859126 -0.59598476]
 [-0.6013868   0.0558806   0.11676984  0.01682443]
 [-0.0280866  -0.56251603 -0.21269545 -0.01356314]]
# Total number of params: 8063
INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:02:35.613547
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <

## Calculating Relative Absolute Error (RAE)

In [6]:
def rae(actual, predicted):
    numerator = np.sum(np.abs(predicted - actual))
    denominator = np.sum(np.abs(np.mean(actual) - actual))
    return numerator / denominator

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = rae(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = rae(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = rae(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Relative Absolute Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Relative Absolute Error p10 Loss": str(p10_loss.mean()),
    "Relative Absolute Error p50 Loss": str(p50_loss.mean()),
    "Relative Absolute Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[ 0.00099336 -0.00099613  0.00099562 -0.0009917 ]
tensor: TemporalFusionTransformer/time_distributed_4/kernel (float32) [20, 4]
[[-0.1872162   0.3253476   0.18593574 -0.49427196]
 [ 0.288401   -0.10835505 -0.20836283  0.49121517]
 [ 0.45712695  0.05524413 -0.33434469 -0.36476237]
 [-0.36678693  0.03381721  0.0718817   0.01069185]
 [-0.01847126 -0.3431983  -0.1306716  -0.00690433]
 [ 0.06607254 -0.29328784 -0.31676498  0.49981037]
 [-0.12950732 -0.1197278   0.27131608  0.181257  ]
 [-0.10162049  0.28506708  0.17781319  0.23296662]
 [ 0.05105373 -0.3895213   0.14780824  0.49005866]
 [ 0.31946108  0.20370983  0.45885578 -0.47607243]
 [ 0.43696368  0.15231526 -0.18435638 -0.49966195]
 [ 0.420306   -0.11672692  0.27148074  0.4141554 ]
 [ 0.0761098   0.2923444  -0.07002746 -0.30980307]
 [ 0.24621247 -0.08138227  0.31635362 -0.29981083]
 [ 0.14472447 -0.33615345 -0.05059061  0.10669195]
 [ 0.3170806  -0.08803189 -0.23570934 -0.3081292 ]
 [ 0.2621921   0.33502015 -0.00651627  0.18761945]
 [-0.

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:14:55.984540
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Mean Absolute Percentage Error ( MAPE )

In [7]:
from sklearn.metrics import mean_absolute_percentage_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_absolute_percentage_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Absolute Percentage Error for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Absolute Percentage Error p10 Loss": str(p10_loss.mean()),
    "Mean Absolute Percentage Error p50 Loss": str(p50_loss.mean()),
    "Mean Absolute Percentage Error p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[[-0.29220834  0.5063342   0.28578264 -0.76463085  0.44604164]
 [-0.16992204 -0.3206005   0.76165247  0.71041226  0.08785115]
 [-0.51889366 -0.5667277  -0.56927806  0.05453968  0.1094387 ]
 [ 0.01572193 -0.02768922 -0.5327429  -0.20069121 -0.01281909]
 [ 0.10420296 -0.45338747 -0.49176925  0.773323   -0.20262453]]
tensor: TemporalFusionTransformer/dense_28/bias (float32) [5]
[ 0.00015795  0.00045845 -0.00072721  0.00051106  0.00045886]
tensor: TemporalFusionTransformer/dense_28/kernel (float32) [5, 5]
[[-0.2918945   0.50529593  0.28721112 -0.7646856   0.4448742 ]
 [-0.16960818 -0.3211609   0.7632186   0.7095658   0.08649015]
 [-0.5189834  -0.5666003  -0.5689789   0.05386044  0.1093748 ]
 [ 0.01543139 -0.02727989 -0.5332826  -0.20116557 -0.01195911]
 [ 0.10403006 -0.45267755 -0.49255213  0.77308345 -0.20204258]]
tensor: TemporalFusionTransformer/dense_4/bias (float32) [1]
[0.]
tensor: TemporalFusionTransformer/dense_4/kernel (float32) [5, 1]
[[-0.37641406]
 [ 0.65268254]
 [ 0.36989117]


INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:28:11.145729
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating R2 Score

In [8]:
from sklearn.metrics import r2_score

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = r2_score(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("R2 Error for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "R2 Error p10 Loss": str(p10_loss.mean()),
    "R2 Error p50 Loss": str(p50_loss.mean()),
    "R2 Error p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:41:20.297815
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Mean Squared Error (MSE)

In [9]:
from sklearn.metrics import mean_squared_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Squared Error p10 Loss": str(p10_loss.mean()),
    "Mean Squared Error p50 Loss": str(p50_loss.mean()),
    "Mean Squared Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 00:53:51.779523
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Root Mean Squared Error (RMSE)

In [10]:
from sklearn.metrics import mean_squared_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    import math
    p10_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast)))
    p50_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast)))
    p90_loss = math.sqrt(mean_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast)))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Root Mean Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss, p50_loss, p90_loss))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Root Mean Squared Error p10 Loss": str(p10_loss),
    "Root Mean Squared Error p50 Loss": str(p50_loss),
    "Root Mean Squared Error p90 Loss": str(p90_loss),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:06:14.340313
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Relative Squared Error (RSE)

In [11]:
def relative_squared_error(true, pred):
    true_mean = np.mean(true)
    squared_error_num = np.sum(np.square(true - pred))
    squared_error_den = np.sum(np.square(true - true_mean))
    rse_loss = squared_error_num / squared_error_den
    return rse_loss

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = relative_squared_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Relative Squared Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Relative Squared Error p10 Loss": str(p10_loss.mean()),
    "Relative Squared Error p50 Loss": str(p50_loss.mean()),
    "Relative Squared Error p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 01:18:44.528783
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Mean Squared Logarithmic Error ( MSLE )

In [13]:
"""
from sklearn.metrics import mean_squared_log_error

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        '''Strips out forecast time and identifier columns.'''
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]

    p10_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Mean Squared Log Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Mean Squared Log Error p10 Loss": str(p10_loss.mean()),
    "Mean Squared Log Error p50 Loss": str(p50_loss.mean()),
    "Mean Squared Log Error p90 Loss": str(p90_loss.mean()),
})
"""

'\nfrom sklearn.metrics import mean_squared_log_error\n\nprint("*** Running tests ***")\ntf.reset_default_graph()\n\nwith tf.Graph().as_default(), tf.Session(config=tf_config) as sess:\n\n    tf.keras.backend.set_session(sess)\n    best_params = opt_manager.get_best_params()\n    model = ModelClass(best_params, use_cudnn=use_gpu)\n\n    model.load(opt_manager.hyperparam_folder)\n\n    print("Computing best validation loss")\n    val_loss = model.evaluate(valid)\n\n    print("Computing test loss")\n    output_map = model.predict(test, return_targets=True)\n    targets = data_formatter.format_predictions(output_map["targets"])\n\n    p10_forecast = data_formatter.format_predictions(output_map["p10"])\n    p50_forecast = data_formatter.format_predictions(output_map["p50"])\n    p90_forecast = data_formatter.format_predictions(output_map["p90"])\n\n    def extract_numerical_data(data):\n        \'\'\'Strips out forecast time and identifier columns.\'\'\'\n        return data[[\n           

## Calculating Root Mean Squared Logarithmic Error ( RMSLE )

In [14]:
"""
from sklearn.metrics import mean_squared_log_error
import math 

print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        '''Strips out forecast time and identifier columns.'''
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p10_forecast)))
    p50_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p50_forecast)))
    p90_loss = math.sqrt(mean_squared_log_error(extract_numerical_data(targets), extract_numerical_data(p90_forecast)))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Root Mean Squared Log Error Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Root Mean Squared Log Error p10 Loss": str(p10_loss.mean()),
    "Root Mean Squared Log Error p50 Loss": str(p50_loss.mean()),
    "Root Mean Squared Log Error p90 Loss": str(p90_loss.mean()),
})
"""

'\nfrom sklearn.metrics import mean_squared_log_error\nimport math \n\nprint("*** Running tests ***")\ntf.reset_default_graph()\n\nwith tf.Graph().as_default(), tf.Session(config=tf_config) as sess:\n\n    tf.keras.backend.set_session(sess)\n    best_params = opt_manager.get_best_params()\n    model = ModelClass(best_params, use_cudnn=use_gpu)\n\n    model.load(opt_manager.hyperparam_folder)\n\n    print("Computing best validation loss")\n    val_loss = model.evaluate(valid)\n\n    print("Computing test loss")\n    output_map = model.predict(test, return_targets=True)\n    targets = data_formatter.format_predictions(output_map["targets"])\n\n    p10_forecast = data_formatter.format_predictions(output_map["p10"])\n    p50_forecast = data_formatter.format_predictions(output_map["p50"])\n    p90_forecast = data_formatter.format_predictions(output_map["p90"])\n\n    def extract_numerical_data(data):\n        \'\'\'Strips out forecast time and identifier columns.\'\'\'\n        return data[

## Calculating Huber Loss, delta = 0.5

In [15]:
def huber_loss(true, pred, delta):
    huber_mse = 0.5 * np.square(true - pred)
    huber_mae = delta * (np.abs(true - pred) - 0.5 * (np.square(delta)))
    return np.where(np.abs(true - pred) <= delta, huber_mse, huber_mae)


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.5)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.5)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.5)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 0.5 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-0.5 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-0.5 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-0.5 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

[ 0.00065205 -0.0001426  -0.00065855 -0.00053915 -0.00059716]
tensor: TemporalFusionTransformer/dense_22/kernel (float32) [5, 5]
[[-0.29136884  0.50544816  0.2864225  -0.76421726  0.44517332]
 [-0.16917744 -0.3214742   0.76211846  0.7094366   0.08697722]
 [-0.5189309  -0.5664482  -0.5703417   0.05330397  0.1094504 ]
 [ 0.01542873 -0.02704871 -0.53360415 -0.20114477 -0.01259346]
 [ 0.10385615 -0.45283782 -0.4925069   0.7725902  -0.20208116]]
tensor: TemporalFusionTransformer/dense_23/bias (float32) [5]
[-0.00040042  0.00061337  0.00062569 -0.00030751 -0.00063063]
tensor: TemporalFusionTransformer/dense_23/kernel (float32) [5, 5]
[[-0.29112116  0.50494856  0.28592956 -0.7643629   0.44603792]
 [-0.16907376 -0.32157576  0.76240283  0.7097422   0.08725736]
 [-0.51913244 -0.5668373  -0.57028043  0.05386046  0.11047085]
 [ 0.01483628 -0.02669022 -0.532771   -0.2007146  -0.01274248]
 [ 0.10367888 -0.45259407 -0.49228925  0.7727026  -0.20220518]]
tensor: TemporalFusionTransformer/dense_24/bias 

[[-0.1872162   0.3253476   0.18593574 -0.49427196]
 [ 0.288401   -0.10835505 -0.20836283  0.49121517]
 [ 0.45712695  0.05524413 -0.33434469 -0.36476237]
 [-0.36678693  0.03381721  0.0718817   0.01069185]
 [-0.01847126 -0.3431983  -0.1306716  -0.00690433]
 [ 0.06607254 -0.29328784 -0.31676498  0.49981037]
 [-0.12950732 -0.1197278   0.27131608  0.181257  ]
 [-0.10162049  0.28506708  0.17781319  0.23296662]
 [ 0.05105373 -0.3895213   0.14780824  0.49005866]
 [ 0.31946108  0.20370983  0.45885578 -0.47607243]
 [ 0.43696368  0.15231526 -0.18435638 -0.49966195]
 [ 0.420306   -0.11672692  0.27148074  0.4141554 ]
 [ 0.0761098   0.2923444  -0.07002746 -0.30980307]
 [ 0.24621247 -0.08138227  0.31635362 -0.29981083]
 [ 0.14472447 -0.33615345 -0.05059061  0.10669195]
 [ 0.3170806  -0.08803189 -0.23570934 -0.3081292 ]
 [ 0.2621921   0.33502015 -0.00651627  0.18761945]
 [-0.20067732 -0.23370181  0.33488742  0.13085502]
 [-0.3475839  -0.02856454  0.19303004  0.45974654]
 [ 0.43719295  0.03098786 -0.28

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 03:14:24.712462
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Huber Loss, delta = 1

In [16]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),1)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),1)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),1)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 1.0 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-1.0 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-1.0 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-1.0 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 06:29:56.838095
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Huber Loss, delta = 5

In [17]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),5)
    p50_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),5)
    p90_loss = huber_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),5)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Huber Loss with Delta 5.0 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Huber Loss Delta with-5.0 p10 Loss": str(p10_loss.mean()),
    "Huber Loss Delta with-5.0 p50 Loss": str(p50_loss.mean()),
    "Huber Loss Delta with-5.0 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 10:28:20.887766
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Calculating Log Cosh Loss

In [18]:
def log_cosh(true, pred):
    logcosh = np.log(np.cosh(pred - true))
    logcosh_loss = np.sum(logcosh)
    return logcosh_loss


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p10_forecast))
    p50_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p50_forecast))
    p90_loss = log_cosh(extract_numerical_data(targets), extract_numerical_data(p90_forecast))

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Log Cosh Loss for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Log Cosh Loss p10 Loss": str(p10_loss.mean()),
    "Log Cosh Loss p50 Loss": str(p50_loss.mean()),
    "Log Cosh Loss p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 12:58:06.712925
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Implementing Quantile Loss , gamma=0.25

In [19]:
def quantile_loss(true, pred, gamma):
    val1 = gamma * np.abs(true - pred)
    val2 = (1-gamma) * np.abs(true - pred)
    q_loss = np.where(true >= pred, val1, val2)
    return q_loss


print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.25)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.25)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.25)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.25 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.25 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.25 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.25 p90 Loss": str(p90_loss.mean()),
})


*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 13:10:16.446187
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Implementing Quantile Loss , gamma=0.5

In [20]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.50)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.50)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.50)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.5 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.5 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.5 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.5 p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 13:27:53.973622
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

## Implementing Quantile Loss , gamma=0.75

In [21]:
print("*** Running tests ***")
tf.reset_default_graph()

with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = data_formatter.format_predictions(output_map["targets"])

    p10_forecast = data_formatter.format_predictions(output_map["p10"])
    p50_forecast = data_formatter.format_predictions(output_map["p50"])
    p90_forecast = data_formatter.format_predictions(output_map["p90"])

    def extract_numerical_data(data):
        """Strips out forecast time and identifier columns."""
        return data[[
            col for col in data.columns
            if col not in {"forecast_time", "identifier"}
        ]]
    
    p10_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p10_forecast),0.75)
    p50_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p50_forecast),0.75)
    p90_loss = quantile_loss(extract_numerical_data(targets), extract_numerical_data(p90_forecast),0.75)

    tf.keras.backend.set_session(default_keras_session)

print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])

print("*"*100)
print("*"*100)
print("Quantile Loss with Gamma 0.75 for Test Data: P10={},  P50={}, P90={}".format(p10_loss.mean(), p50_loss.mean(), p90_loss.mean()))
print("*"*100)
print("*"*100)

loss_experiment_tracker[dataset_folder_path].update({
    "Quantile Loss Gamma with-0.75 p10 Loss": str(p10_loss.mean()),
    "Quantile Loss Gamma with-0.75 p50 Loss": str(p50_loss.mean()),
    "Quantile Loss Gamma with-0.75 p90 Loss": str(p90_loss.mean()),
})

*** Running tests ***
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.3
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = traffic_dataset/saved_models/traffic/fixed
# num_heads = 4
# stack_size = 1
# total_time_steps = 192
# num_encoder_steps = 168
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('categorical_id', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>)]
# input_size = 5
# output_size = 1
# category_count

                                                                  'multiply_1[0][0]']             
                                                                                                  
 activation_1 (Activation)      (None, 1)            0           ['layer_normalization[0][0]']    
                                                                                                  
 layer_normalization_1 (LayerNo  (None, 1, 5)        10          ['add_1[0][0]']                  
 rmalization)                                                                                     
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 1, 1)]      0           ['activation_1[0][0]']           
 sformer/ExpandDims (TensorFlow                                                                   
 OpLayer)                                                                                         
          

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 1)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_9 (Tenso                                  former/stack_1[0][0]']           
 rFlowOpLayer)                                                                                    
                                                                                                  
 add_2 (Add)                    (None, 5)            0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Sum[0][0]',               
                                                                  'multiply_3[0][0]']             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 168, 5, 4)]  0          ['tf_op_layer_TemporalFusionTrans
 sformer/c

                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['time_distributed_27[0][0]',    
 sformer/add_11 (TensorFlowOpLa                                   'time_distributed_28[0][0]']    
 yer)                                                                                             
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_16 (Tens                                  former/strided_slice_10[0][0]']  
 orFlowOpLayer)                                                                                   
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 24, 5)]     0           ['tf_op_layer_TemporalFusionTrans
 sformer/s

 dropout_11 (Dropout)           (None, 24, 5)        0           ['time_distributed_29[0][0]']    
                                                                                                  
 time_distributed_33 (TimeDistr  (None, 24, 5)       30          ['activation_15[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_37 (TimeDistr  (None, 24, 5)       30          ['activation_16[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 time_distributed_41 (TimeDistr  (None, 24, 5)       30          ['activation_17[0][0]']          
 ibuted)                                                                                          
          

...

                                                                 former/Sum_2[0][0]',             
                                                                  'cu_dnnlstm[0][1]',             
                                                                  'cu_dnnlstm[0][2]']             
                                                                                                  
 dense_18 (Dense)               (None, 5)            30          ['activation_4[0][0]']           
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(None, 192, 5)]    0           ['cu_dnnlstm[0][0]',             
 sformer/concat_2 (TensorFlowOp                                   'cu_dnnlstm_1[0][0]']           
 Layer)                                                                                           
                                                                                                  
 dropout_3

                                                                                                  
 layer_normalization_16 (LayerN  (None, 192, 5)      10          ['add_16[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 tf_op_layer_TemporalFusionTran  [(3,)]              0           ['layer_normalization_16[0][0]'] 
 sformer/Shape (TensorFlowOpLay                                                                   
 er)                                                                                              
                                                                                                  
 tf_op_layer_TemporalFusionTran  [()]                0           ['tf_op_layer_TemporalFusionTrans
 sformer/strided_slice_19 (Tens                                  former/Shape[0][0]']             
 orFlowOpL

                                                                                                  
 lambda_10 (Lambda)             (None, None, None)   0           ['tf_op_layer_TemporalFusionTrans
                                                                 former/Cumsum[0][0]']            
                                                                                                  
 add_17 (Add)                   (None, 192, 192)     0           ['lambda[0][0]',                 
                                                                  'lambda_1[0][0]']               
                                                                                                  
 add_18 (Add)                   (None, 192, 192)     0           ['lambda_3[0][0]',               
                                                                  'lambda_4[0][0]']               
                                                                                                  
 add_19 (A

 ibuted)                                                                                          
                                                                                                  
 activation_20 (Activation)     (None, 192, 5)       0           ['time_distributed_53[0][0]']    
                                                                                                  
 time_distributed_54 (TimeDistr  (None, 192, 5)      30          ['activation_20[0][0]']          
 ibuted)                                                                                          
                                                                                                  
 dropout_24 (Dropout)           (None, 192, 5)       0           ['time_distributed_54[0][0]']    
                                                                                                  
 time_distributed_55 (TimeDistr  (None, 192, 5)      30          ['dropout_24[0][0]']             
 ibuted)  

INFO:tensorflow:Restoring parameters from traffic_dataset/saved_models/traffic/fixed/TemporalFusionTransformer.ckpt
Done.
Computing best validation loss
Computing test loss
Training completed @ 2022-09-25 13:46:28.667173
Best validation loss = 2.0376120343109236
Params:
dropout_rate  =  0.3
hidden_layer_size  =  5
learning_rate  =  0.001
max_gradient_norm  =  100.0
minibatch_size  =  128
model_folder  =  traffic_dataset/saved_models/traffic/fixed
num_heads  =  4
stack_size  =  1
total_time_steps  =  192
num_encoder_steps  =  168
num_epochs  =  1
early_stopping_patience  =  5
multiprocessing_workers  =  5
column_definition  =  [('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('hours_from_start', <DataTypes.REAL_VALUED: 0>, <InputTypes.TIME: 5>), ('values', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('time_on_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('day_of_week', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>), ('hours_from_start', <Dat

### Saving Experiments on a json file

In [22]:
with open("traffic_dataset_experiments.json", "w") as outfile:
    json.dump(loss_experiment_tracker, outfile)